<a href="https://colab.research.google.com/github/TimonKazman/DataAnalyticsProjects/blob/main/time_series_lstm_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Timew series analysis using LSTM with Tensorflow and Keras

In [1]:
#importing gpu

!pip install gdown
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 5.9 kB/s 
     |████████████████████████████████| 438 kB 50.3 MB/s 
     |████████████████████████████████| 1.6 MB 54.9 MB/s 
     |████████████████████████████████| 5.8 MB 43.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found ex

In [3]:
#importing packages

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette = 'muted', font_scale=1.5)

rcParams['figure.figsize'] = 22,10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
!

In [4]:
#import dataset

continent = pd.read_csv("csvData.csv")

corona_df = pd.read_csv("covid-hospitalizations.csv")
corona = pd.read_csv("covid-hospitalizations.csv")
corona = corona.merge(continent, left_on="entity", right_on="country", how = "left")

df = corona.copy()

df = df.groupby("date").mean()

corona = corona.groupby(["continent", "date"]).sum()

#set date to index and datetime
corona_df = corona_df.set_index("date")
df.index = pd.to_datetime(df.index)


In [5]:
#split into test and train
train = df.loc[df.index < '2022-01-01']
test = df.loc[df.index >= '2022-01-01']

In [6]:
# create function for feature extraction

def create_feature(df):
    """
    create time series features based on date index
    """
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['month'] =df.index.month
    df['quarter'] =df.index.quarter
    df['year'] =df.index.year
    df['dayofyear'] =df.index.dayofyear

    return df

In [7]:
df = create_feature(df)

In [8]:
df.head()

,value,dayofweek,month,quarter,year,dayofyear
date,,,,,,
2020-01-06,0.219375,0,1,1,2020,6
2020-01-18,0.219375,5,1,1,2020,18
2020-02-12,0.146250,2,2,1,2020,43
2020-02-24,19.441875,0,2,1,2020,55
2020-02-25,15.818333,1,2,1,2020,56
